In [1]:
import os
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchaudio import transforms
from data import CleanSpeechDataset
import time
#from model import Autoencoder
import pdb
import matplotlib.pyplot as plt

In [2]:
import torch
from torch import nn
from torch.autograd import Variable
from torchaudio import transforms
import torch.nn.functional as F
import torchaudio

In [3]:
num_epochs = 50
batch_size = 1024
learning_rate = 1e-3

In [4]:
dataset = CleanSpeechDataset('data/clean/open_slr/', 8192, 50)

In [5]:
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

16384×1, 8192×16, 4096×32,
2048×32, 1024×64, 512×64, 256×128, 128×128, 64×256,
32×256, 16×512, and 8×1024

In [6]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        self.in_dropout = nn.Dropout(p=0.3)
        
        self.conv_1 = nn.Conv1d(1, 16, 32, 2, 15)
        self.norm_1 = nn.PReLU()
        self.conv_2 = nn.Conv1d(16, 32, 32, 2, 15)
        self.norm_2 = nn.PReLU()
        self.conv_3 = nn.Conv1d(32, 32, 32, 2, 15)
        self.norm_3 = nn.PReLU()
        self.conv_4 = nn.Conv1d(32, 64, 32, 2, 15)
        self.norm_4 = nn.PReLU()
        self.conv_5 = nn.Conv1d(64, 64, 32, 2, 15)
        self.norm_5 = nn.PReLU()
        self.conv_6 = nn.Conv1d(64, 128, 32, 2, 15)
        self.norm_6 = nn.PReLU()
        self.conv_7 = nn.Conv1d(128, 128, 32, 2, 15)
        self.norm_7 = nn.PReLU()
        self.conv_8 = nn.Conv1d(128, 256, 32, 2, 15)
        self.norm_8 = nn.PReLU()
        self.conv_9 = nn.Conv1d(256, 256, 32, 2, 15)
        self.norm_9 = nn.PReLU()
        self.conv_10 = nn.Conv1d(256, 512, 32, 2, 15)
        self.norm_10 = nn.PReLU()
        self.conv_11 = nn.Conv1d(512, 1024, 32, 2, 15)
        self.norm_11 = nn.PReLU()
        
        self.deconv_11 = nn.ConvTranspose1d(1024, 512, 32, 2, 15)
        self.norm_d_11 = nn.PReLU()
        self.deconv_10 = nn.ConvTranspose1d(1024, 256, 32, 2, 15)
        self.norm_d_10 = nn.PReLU()
        self.deconv_9 = nn.ConvTranspose1d(512, 256, 32, 2, 15)
        self.norm_d_9 = nn.PReLU()
        self.deconv_8 = nn.ConvTranspose1d(512, 128, 32, 2, 15)
        self.norm_d_8 = nn.PReLU()
        self.deconv_7 = nn.ConvTranspose1d(256, 128, 32, 2, 15)
        self.norm_d_7 = nn.PReLU()
        self.deconv_6 = nn.ConvTranspose1d(256, 64, 32, 2, 15)
        self.norm_d_6 = nn.PReLU()
        self.deconv_5 = nn.ConvTranspose1d(128, 64, 32, 2, 15)
        self.norm_d_5 = nn.PReLU()
        self.deconv_4 = nn.ConvTranspose1d(128, 32, 32, 2, 15)
        self.norm_d_4 = nn.PReLU()
        self.deconv_3 = nn.ConvTranspose1d(64, 32, 32, 2, 15)
        self.norm_d_3 = nn.PReLU()
        self.deconv_2 = nn.ConvTranspose1d(64, 16, 32, 2, 15)
        self.norm_d_2 = nn.PReLU()
        self.deconv_1 = nn.ConvTranspose1d(32, 1, 32, 2, 15)
        self.norm_d_1 = nn.Tanh()

        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d) or isinstance(m, nn.ConvTranspose1d):
                nn.init.xavier_normal_(m.weight.data)

    def forward(self, x):
        c_1 = self.conv_1(x)
        c_2 = self.conv_2(self.norm_1(c_1))
        c_3 = self.conv_3(self.norm_2(c_2))
        c_4 = self.conv_4(self.norm_3(c_3))
        c_5 = self.conv_5(self.norm_4(c_4))
        c_6 = self.conv_6(self.norm_5(c_5))
        c_7 = self.conv_7(self.norm_6(c_6))
        c_8 = self.conv_8(self.norm_7(c_7))
        c_9 = self.conv_9(self.norm_8(c_8))
        c_10 = self.conv_10(self.norm_9(c_9))
        c_11 = self.conv_11(self.norm_10(c_10))
        c_11 = self.norm_11(c_11)

        d_11 = self.deconv_11(c_11)
        pre_d_10 = torch.cat((d_11, c_10), dim=1)
        d_10 = self.norm_d_10(self.deconv_10(pre_d_10))
        pre_d_9 = torch.cat((d_10, c_9), dim=1)
        d_9 = self.norm_d_9(self.deconv_9(pre_d_9))
        pre_d_8 = torch.cat((d_9, c_8), dim=1)
        d_8 = self.norm_d_8(self.deconv_8(pre_d_8))
        pre_d_7 = torch.cat((d_8, c_7), dim=1)
        d_7 = self.norm_d_7(self.deconv_7(pre_d_7))
        pre_d_6 = torch.cat((d_7, c_6), dim=1)
        d_6 = self.norm_d_6(self.deconv_6(pre_d_6))
        pre_d_5 = torch.cat((d_6, c_5), dim=1)
        d_5 = self.norm_d_5(self.deconv_5(pre_d_5))
        pre_d_4 = torch.cat((d_5, c_4), dim=1)
        d_4 = self.norm_d_4(self.deconv_4(pre_d_4))
        pre_d_3 = torch.cat((d_4, c_3), dim=1)
        d_3 = self.norm_d_3(self.deconv_3(pre_d_3))
        pre_d_2 = torch.cat((d_3, c_2), dim=1)
        d_2 = self.norm_d_2(self.deconv_2(pre_d_2))
        pre_d_1 = torch.cat((d_2, c_1), dim=1)
        d_1 = self.norm_d_1(self.deconv_1(pre_d_1))

        return d_1

In [ ]:
model = Autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=1e-5)

model.train()
for epoch in range(num_epochs):
    start = time.time()
    
    for i, data in enumerate(dataloader):
        sample = data
        sample = Variable(sample).cuda()
        output = model(sample)
        loss = criterion(output, sample)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch == 0:
        time_diff = time.time() - start
        print(f'Epoch took {round(time_diff, 4)}s')
        print(f'Expected hours to train: {time_diff * num_epochs / 3600 }')
    if epoch % 5 == 0:
        print(f'epoch [{epoch+1}/{num_epochs}]')
        print(round(loss.item(), 5))

Epoch took 53.683s
Expected hours to train: 0.7455974784162309
epoch [1/50]
1.00555
epoch [6/50]
1.0055


In [ ]:
model

In [ ]:
model.eval()
with torch.no_grad():
    sample = Variable(data[1000]).cuda()
    output = model(sample)
    plt.plot(output.cpu().detach().numpy().T)
    plt.plot(sample.cpu().detach().numpy().T)
#plt.plot(sample.cpu().numpy())

In [ ]:
f = 'data/clean/open_slr/2902-9006-0001.wav'
wave, _ = torchaudio.load(f)
specgram = torchaudio.transforms.Spectrogram(1024, 300)(wave)

print("Shape of spectrogram: {}".format(specgram.size()))

plt.figure()
plt.imshow(specgram.log2()[0,:,:].numpy())